# Task 4

Tras ver los vídeos [My little piece of privacy](https://www.niklasroy.com/project/88/my-little-piece-of-privacy), [Messa di voce](https://youtu.be/GfoqiyB1ndE?feature=shared) y [Virtual air guitar](https://youtu.be/FIAmyoEpV5c?feature=shared) proponer un demostrador reinterpretando la parte de procesamiento de la imagen, tomando como punto de partida alguna de dichas instalaciones.

In [ ]:
# Import necessary libraries
import numpy as np
import cv2

In [ ]:
def highlight_color(frame, color):
    '''
    Function to highlight objects of a specific color in the image.
    Parameters:
    - frame: The input image frame from the video.
    - color: The color to be highlighted (as a string).
    Returns:
    - result: An image with the specified color highlighted.
    '''
    # Convert the image from BGR (OpenCV's default) to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Retrieve the lower and upper HSV range for the selected color from the colors dictionary
    lower_range, upper_range = colors[color]

    # Create a binary mask where the selected color is within the specified range
    mask = cv2.inRange(hsv, np.array(lower_range), np.array(upper_range))

    # Use the mask to extract the colored objects from the original frame
    result = cv2.bitwise_and(frame, frame, mask=mask)

    return result

def generate_trail(current_frame, trail_frames):
    '''
    Function to generate a trail of frames for visual effect.
    Parameters:
    - current_frame: The most recent highlighted frame.
    - trail_frames: A list of previous frames to create the trail effect.
    Returns:
    - trail_image: An image that shows the current frame with a trail effect.
    '''
    # Initialize an image to hold the accumulated trail frames, starting as a black image
    trail_image = np.zeros_like(current_frame)

    # Blend the current frame into the trail image
    trail_image = cv2.addWeighted(current_frame, 0.8, trail_image, 0.2, 0)

    # Blend in previous frames to create a trail effect
    for i in range(len(trail_frames)):
        if len(trail_frames) > i:
            trail_frame = trail_frames[i]  # Get the historical frame
            trail_image = cv2.addWeighted(trail_frame, 0.5, trail_image, 0.5, 0)  # Blend the frames

    return trail_image

# Dictionary of available colors and their HSV ranges for color detection
colors = {
    'White': ([0, 0, 200], [360, 40, 255]),
    'Red': ([0, 100, 100], [10, 255, 255]),
    'Green': ([35, 100, 100], [85, 255, 255]),
    'Blue': ([85, 100, 100], [140, 255, 255]),
    'Yellow': ([20, 100, 100], [30, 255, 255]),
    'Orange': ([10, 100, 100], [20, 255, 255]),
    'Purple': ([130, 100, 100], [160, 255, 255]),
    'Pink': ([160, 100, 100], [200, 255, 255]),
    'Brown': ([10, 50, 50], [30, 255, 100]),
    'Gray': ([0, 0, 50], [360, 40, 70])
}

# Instructions for the user on which keys to press for different colors
text = ["Buttons:",
        " 'r' for Red,",
        " 'g' for Green,",
        " 'b' for Blue,",
        " 'y' for Yellow,",
        " 'o' for Orange,",
        " 'p' for Purple,",
        " 'n' for Pink,",
        " 'z' for Brown,",
        " 'x' for Gray,",
        " 'w' for White",
        " 'q' to Quit"
]

# Initialize the selected color to Red
selected_color = 'Red'

# Initialize video capture from the default camera (0)
cap = cv2.VideoCapture(0)

ret, frame = cap.read()  # Read the first frame from the camera

# Initialize a list to keep the last 20 frames for trail generation
trail_frames = [np.zeros_like(frame) for _ in range(20)]  # Create an array of black frames

while True:    
    ret, frame = cap.read()  # Read a new frame from the camera

    # Create a blank image to display information on
    infoFrame = np.zeros((25 * len(text), 400, 3), dtype=np.uint8)  # Height based on number of text lines

    # Positioning of the text within the information frame
    y_position = 20
    for t in text:
        # Draw the text on the infoFrame
        cv2.putText(infoFrame, t, (10, y_position), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        y_position += 20  # Increment y_position for the next line

    # Show the information frame in a window titled 'Commands Information'
    cv2.imshow('Commands Information', infoFrame)

    if ret:
        # Call the function to highlight the selected color in the current frame
        highlighted_frame = highlight_color(frame, selected_color)

        # Update the trail_frames list by adding the highlighted frame and removing the oldest frame
        trail_frames = [highlighted_frame] + trail_frames[:-1]

        # Generate the image with the trail effect
        trail_image = generate_trail(highlighted_frame, trail_frames)

        # Show the resulting frame with the trail effect in a window titled 'Color Detection'
        cv2.imshow('Color Detection', trail_image)

    # Wait for a key press and capture the key pressed
    key = cv2.waitKey(1) & 0xFF

    # Check if the pressed key corresponds to a color change
    if key == ord('r'):
        selected_color = 'Red'
    elif key == ord('g'):
        selected_color = 'Green'
    elif key == ord('b'):
        selected_color = 'Blue'
    elif key == ord('y'):
        selected_color = 'Yellow'
    elif key == ord('o'):
        selected_color = 'Orange'
    elif key == ord('p'):
        selected_color = 'Purple'
    elif key == ord('n'):
        selected_color = 'Pink'
    elif key == ord('z'):
        selected_color = 'Brown'
    elif key == ord('x'):
        selected_color = 'Gray'
    elif key == ord('w'):
        selected_color = 'White'
    elif key == ord('q'):  # If 'q' is pressed, exit the loop
        break

# Release the camera and close all OpenCV windows when done
cap.release()
cv2.destroyAllWindows()